In [ ]:
!pip install datasets
!pip install llama_index
!pip install sentence-transformers
!pip install datasets llama-index matplotlib numpy
!pip install --upgrade llama-index
!pip install datasets llama-index-core llama-index-embeddings-huggingface matplotlib numpy sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
!pip install -U langchain-community
!pip install llama-index-llms-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00


In [ ]:
import datasets
from datasets import load_dataset
from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
# Configurer LlamaIndex pour utiliser un modèle d'embedding local
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Charger le jeu de données MNIST depuis Hugging Face
dataset = load_dataset("mnist")

# Préparer les données pour LlamaIndex
train_data = dataset['train']
test_data = dataset['test']



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/15.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
# Créer des documents pour LlamaIndex basés sur les métadonnées
documents = []
for i, example in enumerate(train_data):
    label = example['label']
    # Créer un document avec le label comme contenu textuel
    doc = Document(
        text=f"Image of digit {label}",
        metadata={"index": i, "label": label}
    )
    documents.append(doc)

# Créer un index avec LlamaIndex
index = VectorStoreIndex.from_documents(documents)

# Explorer la structure des données
print(f"Nombre de documents indexés: {len(documents)}")
print(f"Nombre d'exemples d'entraînement: {len(train_data)}")
print(f"Nombre d'exemples de test: {len(test_data)}")



Nombre de documents indexés: 60000
Nombre d'exemples d'entraînement: 60000
Nombre d'exemples de test: 10000


In [ ]:
# Fonction pour afficher une image MNIST
def display_image(example, index):
    image = np.array(example['image'])
    label = example['label']

    plt.figure(figsize=(4, 4))
    plt.imshow(image, cmap='gray')
    plt.title(f"Chiffre: {label}")
    plt.axis('off')
    plt.savefig(f'mnist_example_{index}.png')
    plt.close()

# Afficher les 5 premières images du jeu d'entraînement
for i in range(5):
    display_image(train_data[i], i)



In [ ]:
# Prétraitement : normalisation des pixels
def normalize_images(example):
    example['image'] = np.array(example['image']) / 255.0
    return example

# Appliquer la normalisation au dataset
train_data_normalized = train_data.map(normalize_images)
test_data_normalized = test_data.map(normalize_images)



Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [ ]:
# Vérifier les valeurs normalisées
sample_image = np.array(train_data_normalized[0]['image'])
print(f"Valeurs min/max après normalisation: {sample_image.min():.2f}/{sample_image.max():.2f}")

# Exemple de filtrage avec LlamaIndex : rechercher les images du chiffre 5
query_engine = index.as_query_engine()
response = query_engine.query("Find images of digit 5")

In [ ]:
# Extraire les indices des documents correspondant au chiffre 5
fives_indices = []
for node in response.source_nodes:
    if node.metadata['label'] == 5:
        fives_indices.append(node.metadata['index'])

print(f"Nombre d'images du chiffre 5: {len(fives_indices)}")

# Sauvegarder les indices des images du chiffre 5 localement (optionnel)
with open("mnist_fives_only.txt", "w") as f:
    for idx in fives_indices:
        f.write(f"{idx}\n")